---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [11]:
import pandas as pd

def date_sorter():
    
    # Your code here
    my_list = [9, 84, 2, 53, 28, 474, 153, 13, 129, 98, 111, 225, 31, 171, 191, 486, 335, 415, 36, 405, 323, 422, 375, 380, 345, 57, 481, 436, 104, 299, 162, 154, 402, 95, 73, 108, 156, 332, 182, 82, 351, 278, 214, 155, 223, 473, 49, 317, 11, 319, 40, 418, 165, 382, 370, 3, 50, 363, 219, 465, 237, 23, 342, 204, 258, 315, 27, 93, 17, 303, 488, 283, 395, 309, 419, 123, 19, 117, 232, 72, 189, 369, 493, 318, 239, 148, 105, 336, 6, 200, 81, 65, 434, 164, 378, 313, 495, 424, 398, 5, 254, 296, 75, 167, 21, 259, 499, 347, 150, 78, 340, 441, 361, 267, 221, 466, 134, 39, 197, 355, 430, 80, 444, 246, 85, 215, 263, 74, 403, 458, 16, 25, 127, 454, 70, 44, 59, 103, 112, 429, 88, 179, 358, 470, 205, 294, 397, 137, 295, 35, 438, 247, 209, 61, 107, 285, 175, 99, 455, 24, 421, 275, 48, 426, 489, 136, 30, 274, 10, 178, 1, 280, 447, 185, 228, 135, 69, 492, 199, 352, 8, 276, 230, 334, 96, 38, 368, 261, 404, 168, 29, 437, 423, 54, 284, 485, 68, 32, 349, 41, 63, 416, 55, 130, 116, 76, 462, 330, 37, 256, 390, 216, 174, 180, 476, 312, 265, 115, 71, 218, 202, 440, 385, 373, 210, 89, 149, 26, 7, 435, 482, 177, 157, 412, 22, 194, 14, 151, 233, 206, 245, 122, 94, 461, 226, 97, 91, 51, 33, 453, 67, 46, 322, 66, 487, 399, 138, 62, 211, 52, 269, 119, 100, 442, 310, 143, 301, 113, 298, 331, 478, 272, 354, 0, 249, 192, 86, 172, 357, 450, 300, 477, 163, 308, 196, 47, 133, 359, 64, 42, 409, 406, 483, 238, 193, 311, 140, 388, 56, 372, 236, 110, 248, 60, 181, 203, 326, 90, 169, 292, 479, 142, 4, 124, 324, 121, 131, 166, 468, 365, 213, 87, 353, 101, 333, 114, 459, 45, 338, 18, 222, 343, 20, 224, 12, 79, 387, 251, 120, 471, 77, 376, 327, 432, 384, 321, 212, 407, 266, 145, 201, 456, 260, 305, 420, 329, 392, 417, 190, 158, 443, 83, 374, 457, 125, 328, 159, 195, 147, 377, 367, 394, 494, 304, 446, 43, 262, 128, 102, 449, 184, 469, 452, 234, 362, 356, 144, 484, 291, 188, 414, 92, 241, 306, 350, 425, 281, 207, 126, 302, 146, 451, 498, 339, 250, 344, 346, 348, 496, 106, 118, 270, 433, 307, 173, 410, 252, 314, 490, 337, 277, 391, 325, 264, 289, 160, 341, 132, 428, 497, 445, 187, 183, 271, 396, 293, 400, 360, 297, 491, 371, 389, 386, 288, 379, 268, 472, 273, 287, 448, 176, 411, 408, 242, 364, 58, 467, 170, 15, 240, 316, 229, 217, 109, 227, 290, 460, 393, 282, 34, 220, 243, 208, 139, 320, 383, 244, 286, 480, 431, 279, 198, 381, 463, 366, 255, 439, 401, 475, 257, 152, 235, 464, 253, 231, 427, 141, 186, 161, 413]
    
    return pd.Series(my_list, dtype="int32")